In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import warnings
warnings.filterwarnings('ignore')

# Tentar diferentes abordagens para carregar o CSV problemático
print("Tentando carregar o dataset...")

# Método 1: Tentar com error_bad_lines para pular linhas problemáticas
try:
    df = pd.read_csv('csv_longo/projetos_resultados_pessoas_valores.csv', 
                     encoding='UTF-8', 
                     low_memory=False,
                     sep=';',  # Garantir que está usando vírgula
                     quotechar='"')  # Tratar aspas
    print("✓ Dados carregados")
except Exception as e:
    print(f"Erro no método 1: {e}")
    
    # Método 2: Ler linha por linha para identificar o problema
    try:
        # Primeiro, descobrir quantas colunas deveria ter
        with open('csv_longo/projetos_resultados_pessoas_valores.csv', 'r', encoding='latin-1') as f:
            header = f.readline()
            n_cols = len(header.split(','))
            print(f"Número esperado de colunas: {n_cols}")
        
        # Carregar pulando linhas ruins
        df = pd.read_csv('csv_longo/projetos_resultados_pessoas_valores.csv',
                        encoding='latin-1',
                        low_memory=False,
                        on_bad_lines='warn',
                        engine='python',
                        sep=',',
                        quotechar='"',
                        escapechar='\\')
        print("✓ Dados carregados com engine='python'")
    except:
        # Método 3: Última tentativa - ler em chunks
        chunks = []
        chunk_size = 10000
        problematic_lines = []
        
        for i, chunk in enumerate(pd.read_csv('csv_longo/projetos_resultados_pessoas_valores.csv',
                                             encoding='latin-1',
                                             low_memory=False,
                                             on_bad_lines='skip',
                                             chunksize=chunk_size)):
            chunks.append(chunk)
            print(f"  Chunk {i+1} carregado ({len(chunk)} linhas)")
            
        df = pd.concat(chunks, ignore_index=True)
        print("✓ Dados carregados em chunks")

print("\n" + "="*80)
print("DADOS CARREGADOS COM SUCESSO")
print("="*80)
print(f"Total de registros: {len(df):,}")
print(f"Total de colunas: {df.shape[1]}")

# ============================================================
# FILTRO CRÍTICO: REMOVER DADOS DE 2023 (DADOS INCOMPLETOS)
# ============================================================
print("\n" + "="*80)
print("APLICANDO FILTRO CRÍTICO - REMOVENDO 2023")
print("="*80)

# Verificar distribuição antes do filtro
if 'ano_referencia' in df.columns:
    print(f"Registros por ano ANTES do filtro:")
    print(df['ano_referencia'].value_counts().sort_index())
    
    # Contagem antes
    total_antes = len(df)
    registros_2023 = (df['ano_referencia'] == 2023).sum()
    
    # APLICAR FILTRO - REMOVER 2023
    df = df[df['ano_referencia'] != 2023].copy()
    
    print(f"\n✓ Removidos {registros_2023:,} registros de 2023 ({registros_2023/total_antes*100:.1f}%)")
    print(f"✓ Dataset filtrado: {len(df):,} registros (2018-2022)")
    
    print(f"\nRegistros por ano APÓS filtro:")
    print(df['ano_referencia'].value_counts().sort_index())
else:
    print("⚠️ Coluna 'ano_referencia' não encontrada - verificar estrutura dos dados")

print("="*80)

# Verificar colunas esperadas
colunas_esperadas = ['id_projeto', 'ano_referencia', 'empresa_razao_social', 'cnpj', 
                     'nome_projeto', 'descricao_projeto', 'tipo_projeto', 'natureza',
                     'elemento_tecnologico', 'desafio_tecnologico', 'metodologia',
                     'setor_analise', 'ciclo_maior_1_ano']

colunas_faltando = [col for col in colunas_esperadas if col not in df.columns]
if colunas_faltando:
    print(f"\n⚠️ Colunas faltando: {colunas_faltando}")
    print("Colunas disponíveis:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:3d}. {col}")
else:
    print("✓ Todas as colunas esperadas encontradas")

# Continuar com a análise se temos as colunas principais
if all(col in df.columns for col in ['cnpj', 'ano_referencia', 'nome_projeto']):
    print(f"\nEmpresas únicas: {df['cnpj'].nunique():,}")
    print(f"Anos disponíveis: {sorted(df['ano_referencia'].dropna().unique())}")
    
    # Análise de projetos multianuais
    if 'ciclo_maior_1_ano' in df.columns:
        df['projeto_multianual'] = df['ciclo_maior_1_ano'] == 'Sim'
        print(f"\nProjetos multianuais: {df['projeto_multianual'].sum():,} ({df['projeto_multianual'].mean()*100:.1f}%)")
    else:
        print("\n⚠️ Coluna 'ciclo_maior_1_ano' não encontrada")
    
    # Criar identificador único para rastrear projetos entre anos
    df['projeto_chave'] = df['cnpj'].astype(str) + '_' + df['nome_projeto'].fillna('').astype(str).str[:50]
    
    # Distribuição por setor
    if 'setor_analise' in df.columns:
        print("\nDistribuição por setor:")
        print(df['setor_analise'].value_counts())
    else:
        print("\n⚠️ Coluna 'setor_analise' não encontrada")
    
    # Verificar anos com dados
    print("\nProjetos por ano:")
    print(df['ano_referencia'].value_counts().sort_index())
else:
    print("\n❌ Colunas essenciais não encontradas. Verifique o arquivo CSV.")

# Mostrar informações sobre o dataset
print("\n" + "="*80)
print("INFORMAÇÕES DO DATASET")
print("="*80)
print(df.info())

# Verificar primeiras linhas
print("\nPrimeiras 3 linhas do dataset:")
print(df.head(3))

Tentando carregar o dataset...
✓ Dados carregados

DADOS CARREGADOS COM SUCESSO
Total de registros: 74,502
Total de colunas: 26

APLICANDO FILTRO CRÍTICO - REMOVENDO 2023
Registros por ano ANTES do filtro:
ano_referencia
2018    10876
2019    12168
2020    11660
2021    13198
2022    13786
2023    12814
Name: count, dtype: int64

✓ Removidos 12,814 registros de 2023 (17.2%)
✓ Dataset filtrado: 61,688 registros (2018-2022)

Registros por ano APÓS filtro:
ano_referencia
2018    10876
2019    12168
2020    11660
2021    13198
2022    13786
Name: count, dtype: int64
✓ Todas as colunas esperadas encontradas

Empresas únicas: 4,885
Anos disponíveis: [np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022)]

Projetos multianuais: 46,238 (75.0%)

Distribuição por setor:
setor_analise
TIC                          16272
Química e Farmácia           11708
Mecânica e Transporte         7990
Agroindústria e Alimentos     7499
Transversal                   7090
Eletroeletrônic

In [2]:
# ============================================================
# CHUNK 2: ANÁLISE DEFINITIVA COM GRANITE - THRESHOLD 0.85
# ============================================================

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("ANÁLISE DE SIMILARIDADE COM GRANITE IBM - THRESHOLD 0.85")
print("="*80)

# Filtrar dados
print("\nFiltrando dados de Eletroeletrônica - 2021 - Processo...")
filtro = (
    (df['setor_analise'] == 'Eletroeletrônica') & 
    (df['ano_referencia'] == 2021) & 
    (df['natureza'] == 'Processo')
)

eletr_2021_proc = df[filtro].copy()
print(f"✓ Projetos filtrados: {len(eletr_2021_proc)}")

# ============================================================
# PREPARAÇÃO DOS TEXTOS
# ============================================================

print("\n" + "="*80)
print("PREPARAÇÃO DOS DADOS")
print("="*80)

# Campos de texto principais
campos_texto = ['nome_projeto', 'elemento_tecnologico', 'desafio_tecnologico', 
                'metodologia', 'descricao_projeto']

# Limpar valores nulos
for campo in campos_texto:
    if campo in eletr_2021_proc.columns:
        eletr_2021_proc[campo] = eletr_2021_proc[campo].fillna('').astype(str)

# Criar texto completo sem truncamento
def criar_texto_completo(row):
    """Concatena todos os campos disponíveis"""
    partes = []
    for campo in campos_texto:
        if campo in row and row[campo] and str(row[campo]).strip():
            partes.append(str(row[campo]).strip())
    return " ".join(partes)

eletr_2021_proc['texto_completo'] = eletr_2021_proc.apply(criar_texto_completo, axis=1)

# Estatísticas dos textos
comprimentos = eletr_2021_proc['texto_completo'].str.len()
print(f"\nEstatísticas dos textos:")
print(f"  - Total de projetos: {len(eletr_2021_proc)}")
print(f"  - Comprimento médio: {comprimentos.mean():.0f} caracteres")
print(f"  - Comprimento mínimo: {comprimentos.min():.0f} caracteres")
print(f"  - Comprimento máximo: {comprimentos.max():.0f} caracteres")

# Verificar e remover textos vazios
textos_vazios = (comprimentos == 0).sum()
if textos_vazios > 0:
    print(f"  ⚠️ Removendo {textos_vazios} projetos sem texto")
    eletr_2021_proc = eletr_2021_proc[comprimentos > 0].copy()

# ============================================================
# GERAR EMBEDDINGS COM GRANITE
# ============================================================

print("\n" + "="*80)
print("GERANDO EMBEDDINGS COM GRANITE IBM")
print("="*80)

# Carregar modelo Granite
print("Carregando modelo ibm-granite/granite-embedding-278m-multilingual...")
try:
    model = SentenceTransformer('ibm-granite/granite-embedding-278m-multilingual')
    print("✓ Modelo Granite carregado com sucesso")
except:
    print("⚠️ Usando modelo alternativo multilíngue...")
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    print("✓ Modelo alternativo carregado")

# Preparar textos
textos = eletr_2021_proc['texto_completo'].tolist()
ids_projetos = eletr_2021_proc['id_projeto'].tolist() if 'id_projeto' in eletr_2021_proc.columns else eletr_2021_proc.index.tolist()
nomes_projetos = eletr_2021_proc['nome_projeto'].tolist()

print(f"\nGerando embeddings para {len(textos)} projetos...")

# Gerar embeddings
embeddings = model.encode(
    textos,
    batch_size=16,
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=True
)

print(f"✓ Embeddings gerados: shape {embeddings.shape}")

# ============================================================
# CALCULAR MATRIZ DE SIMILARIDADE
# ============================================================

print("\n" + "="*80)
print("CALCULANDO SIMILARIDADES")
print("="*80)

# Calcular matriz de similaridade
print("Calculando matriz de similaridade...")
similarity_matrix = cosine_similarity(embeddings)

# Estatísticas gerais
upper_triangle = np.triu(similarity_matrix, k=1)
all_similarities = upper_triangle[upper_triangle > 0]

print(f"\n✓ Matriz calculada: {similarity_matrix.shape}")
print(f"  - Total de comparações únicas: {len(all_similarities):,}")
print(f"\nEstatísticas de similaridade:")
print(f"  - Média: {all_similarities.mean():.4f}")
print(f"  - Desvio padrão: {all_similarities.std():.4f}")
print(f"  - Mínima: {all_similarities.min():.4f}")
print(f"  - Máxima: {all_similarities.max():.4f}")
print(f"  - Mediana: {np.median(all_similarities):.4f}")

# Percentis importantes
percentis = [75, 80, 85, 90, 95, 99]
print(f"\nPercentis:")
for p in percentis:
    valor = np.percentile(all_similarities, p)
    print(f"  - {p}%: {valor:.4f}")

# ============================================================
# IDENTIFICAR PARES SIMILARES (>= 0.85)
# ============================================================

print("\n" + "="*80)
print("IDENTIFICANDO PARES SIMILARES (THRESHOLD >= 0.85)")
print("="*80)

threshold = 0.85
pares_similares = []

# Iterar sobre triangular superior para evitar duplicatas
print("Procurando pares com similaridade >= 0.85...")
for i in range(len(ids_projetos)):
    for j in range(i + 1, len(ids_projetos)):
        similaridade = similarity_matrix[i, j]
        
        if similaridade >= threshold:
            # Coletar informações adicionais se disponíveis
            registro = {
                'IdProjeto_1': ids_projetos[i],
                'NomeProjeto_1': nomes_projetos[i],
                'IdProjeto_2': ids_projetos[j],
                'NomeProjeto_2': nomes_projetos[j],
                'Similaridade_score': float(similaridade)
            }
            
            # Adicionar informações de empresa se disponíveis
            if 'empresa_razao_social' in eletr_2021_proc.columns:
                registro['Empresa_1'] = eletr_2021_proc.iloc[i]['empresa_razao_social']
                registro['Empresa_2'] = eletr_2021_proc.iloc[j]['empresa_razao_social']
                registro['Mesma_Empresa'] = registro['Empresa_1'] == registro['Empresa_2']
            
            if 'cnpj' in eletr_2021_proc.columns:
                registro['CNPJ_1'] = eletr_2021_proc.iloc[i]['cnpj']
                registro['CNPJ_2'] = eletr_2021_proc.iloc[j]['cnpj']
            
            pares_similares.append(registro)

# Criar DataFrame com pares similares
eletr_2021_proc_similar = pd.DataFrame(pares_similares)

if len(eletr_2021_proc_similar) > 0:
    # Ordenar por similaridade
    eletr_2021_proc_similar = eletr_2021_proc_similar.sort_values('Similaridade_score', ascending=False)
    
    print(f"\n✓ Pares encontrados: {len(eletr_2021_proc_similar)}")
    
    # Estatísticas dos pares
    print(f"\nEstatísticas dos pares similares:")
    print(f"  - Similaridade máxima: {eletr_2021_proc_similar['Similaridade_score'].max():.4f}")
    print(f"  - Similaridade média: {eletr_2021_proc_similar['Similaridade_score'].mean():.4f}")
    print(f"  - Similaridade mínima: {eletr_2021_proc_similar['Similaridade_score'].min():.4f}")
    
    # Análise por faixas
    print(f"\nDistribuição por faixas:")
    faixas = [(0.98, 1.00), (0.95, 0.98), (0.90, 0.95), (0.85, 0.90)]
    for min_val, max_val in faixas:
        if max_val == 1.00:
            count = (eletr_2021_proc_similar['Similaridade_score'] >= min_val).sum()
        else:
            count = ((eletr_2021_proc_similar['Similaridade_score'] >= min_val) & 
                    (eletr_2021_proc_similar['Similaridade_score'] < max_val)).sum()
        pct = count / len(eletr_2021_proc_similar) * 100 if len(eletr_2021_proc_similar) > 0 else 0
        print(f"  - [{min_val:.2f}, {max_val:.2f}): {count} pares ({pct:.1f}%)")
    
    # Top 10 pares mais similares
    print(f"\nTop 10 pares mais similares:")
    print("-" * 80)
    for idx, row in eletr_2021_proc_similar.head(10).iterrows():
        print(f"\nSimilaridade: {row['Similaridade_score']:.4f}")
        print(f"  Projeto 1: [{row['IdProjeto_1']}] {row['NomeProjeto_1'][:50]}...")
        print(f"  Projeto 2: [{row['IdProjeto_2']}] {row['NomeProjeto_2'][:50]}...")
        if 'Mesma_Empresa' in row:
            tipo = "MESMA EMPRESA" if row['Mesma_Empresa'] else "EMPRESAS DIFERENTES"
            print(f"  Tipo: {tipo}")
    
    # Análise por empresa se disponível
    if 'Mesma_Empresa' in eletr_2021_proc_similar.columns:
        mesma_empresa = eletr_2021_proc_similar['Mesma_Empresa'].sum()
        diferentes = len(eletr_2021_proc_similar) - mesma_empresa
        print(f"\n" + "="*80)
        print("ANÁLISE POR TIPO DE PAR")
        print("="*80)
        print(f"  - Pares da mesma empresa: {mesma_empresa} ({mesma_empresa/len(eletr_2021_proc_similar)*100:.1f}%)")
        print(f"  - Pares entre empresas diferentes: {diferentes} ({diferentes/len(eletr_2021_proc_similar)*100:.1f}%)")
        
        # Empresas com mais pares similares
        if 'Empresa_1' in eletr_2021_proc_similar.columns:
            print(f"\nEmpresas com mais projetos similares:")
            todas_empresas = pd.concat([
                eletr_2021_proc_similar['Empresa_1'],
                eletr_2021_proc_similar['Empresa_2']
            ])
            top_empresas = todas_empresas.value_counts().head(5)
            for empresa, count in top_empresas.items():
                print(f"  - {empresa[:50]}: {count} aparições em pares similares")
else:
    print(f"\n⚠️ Nenhum par encontrado com similaridade >= {threshold}")

# ============================================================
# SALVAR RESULTADOS EM CSV
# ============================================================

print("\n" + "="*80)
print("SALVANDO RESULTADOS")
print("="*80)

# Remover coluna temporária antes de salvar
if 'texto_completo' in eletr_2021_proc.columns:
    eletr_2021_proc_export = eletr_2021_proc.drop(columns=['texto_completo'])
else:
    eletr_2021_proc_export = eletr_2021_proc.copy()

# Salvar dataset filtrado
arquivo_projetos = 'eletr_2021_proc.csv'
eletr_2021_proc_export.to_csv(arquivo_projetos, index=False, encoding='utf-8-sig', sep=';')
print(f"✓ Dataset filtrado salvo: {arquivo_projetos}")
print(f"  - Total de projetos: {len(eletr_2021_proc_export)}")
print(f"  - Total de colunas: {eletr_2021_proc_export.shape[1]}")

# Salvar pares similares
if len(eletr_2021_proc_similar) > 0:
    # Selecionar colunas essenciais para o CSV
    colunas_essenciais = ['IdProjeto_1', 'NomeProjeto_1', 'IdProjeto_2', 'NomeProjeto_2', 'Similaridade_score']
    colunas_extras = ['Empresa_1', 'Empresa_2', 'Mesma_Empresa', 'CNPJ_1', 'CNPJ_2']
    
    # Adicionar colunas extras se existirem
    colunas_export = colunas_essenciais + [col for col in colunas_extras if col in eletr_2021_proc_similar.columns]
    eletr_2021_proc_similar_export = eletr_2021_proc_similar[colunas_export]
    
    arquivo_similares = 'eletr_2021_proc_similar.csv'
    eletr_2021_proc_similar_export.to_csv(arquivo_similares, index=False, encoding='utf-8-sig', sep=';')
    print(f"\n✓ Pares similares salvos: {arquivo_similares}")
    print(f"  - Total de pares: {len(eletr_2021_proc_similar_export)}")
    print(f"  - Colunas incluídas: {', '.join(colunas_export)}")
else:
    # Criar arquivo vazio com estrutura básica
    df_vazio = pd.DataFrame(columns=['IdProjeto_1', 'NomeProjeto_1', 'IdProjeto_2', 'NomeProjeto_2', 'Similaridade_score'])
    df_vazio.to_csv('eletr_2021_proc_similar.csv', index=False, encoding='utf-8-sig', sep=';')
    print(f"\n✓ Arquivo de pares similares criado (vazio - nenhum par acima do threshold)")

# ============================================================
# RESUMO FINAL
# ============================================================

print("\n" + "="*80)
print("ANÁLISE CONCLUÍDA COM SUCESSO")
print("="*80)

print(f"\n📊 RESUMO EXECUTIVO:")
print(f"  • Modelo utilizado: Granite IBM Multilingual")
print(f"  • Projetos analisados: {len(eletr_2021_proc)}")
print(f"  • Threshold aplicado: {threshold}")
print(f"  • Pares similares identificados: {len(eletr_2021_proc_similar)}")

if len(eletr_2021_proc_similar) > 0:
    taxa = len(eletr_2021_proc_similar) / (len(eletr_2021_proc) * (len(eletr_2021_proc) - 1) / 2) * 100
    print(f"  • Taxa de pares similares: {taxa:.3f}%")

print(f"\n📁 Arquivos gerados:")
print(f"  1. eletr_2021_proc.csv - Base completa filtrada")
print(f"  2. eletr_2021_proc_similar.csv - Pares com similaridade >= 0.85")

2025-11-19 11:07:04.020789: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ANÁLISE DE SIMILARIDADE COM GRANITE IBM - THRESHOLD 0.85

Filtrando dados de Eletroeletrônica - 2021 - Processo...
✓ Projetos filtrados: 366

PREPARAÇÃO DOS DADOS

Estatísticas dos textos:
  - Total de projetos: 366
  - Comprimento médio: 8058 caracteres
  - Comprimento mínimo: 1077 caracteres
  - Comprimento máximo: 16166 caracteres

GERANDO EMBEDDINGS COM GRANITE IBM
Carregando modelo ibm-granite/granite-embedding-278m-multilingual...
✓ Modelo Granite carregado com sucesso

Gerando embeddings para 366 projetos...


Batches:   0%|          | 0/23 [00:00<?, ?it/s]

✓ Embeddings gerados: shape (366, 768)

CALCULANDO SIMILARIDADES
Calculando matriz de similaridade...

✓ Matriz calculada: (366, 366)
  - Total de comparações únicas: 66,795

Estatísticas de similaridade:
  - Média: 0.6096
  - Desvio padrão: 0.0578
  - Mínima: 0.4090
  - Máxima: 1.0000
  - Mediana: 0.6069

Percentis:
  - 75%: 0.6459
  - 80%: 0.6556
  - 85%: 0.6678
  - 90%: 0.6831
  - 95%: 0.7071
  - 99%: 0.7563

IDENTIFICANDO PARES SIMILARES (THRESHOLD >= 0.85)
Procurando pares com similaridade >= 0.85...

✓ Pares encontrados: 70

Estatísticas dos pares similares:
  - Similaridade máxima: 1.0000
  - Similaridade média: 0.9478
  - Similaridade mínima: 0.8500

Distribuição por faixas:
  - [0.98, 1.00): 33 pares (47.1%)
  - [0.95, 0.98): 10 pares (14.3%)
  - [0.90, 0.95): 8 pares (11.4%)
  - [0.85, 0.90): 19 pares (27.1%)

Top 10 pares mais similares:
--------------------------------------------------------------------------------

Similaridade: 1.0000
  Projeto 1: [68931] Torre Móvel - D

In [1]:
from leann import LeannBuilder
from pathlib import Path
from llama_index.core import SimpleDirectoryReader

# Defina onde seu índice será salvo
INDEX_PATH = "./leann_index.leann/leann_index"

# Crie o construtor usando Ollama para embeddings
builder = LeannBuilder(
    backend_name="hnsw",                # Motor de busca (padrão, mais leve)
    embedding_mode="ollama",            # Usar Ollama para embeddings
    embedding_model="nomic-embed-text"  # Modelo de embedding via Ollama
)

# Adicione seus documentos
pasta_documentos = Path("./leann_index")

# Usando SimpleDirectoryReader para detecção automática de tipos
reader = SimpleDirectoryReader(
    input_dir=pasta_documentos,
    recursive=True,  # Processa subpastas também
    required_exts=['.pdf', '.txt', '.md', '.docx']  # Filtra apenas esses tipos
)

# Carrega os documentos
documentos = reader.load_data()
print(f"📄 Encontrados {len(documentos)} documentos")


# Adiciona cada documento ao builder
for i, doc in enumerate(documentos, 1):
    builder.add_text(
        text=doc.text,
        metadata={
            "source": doc.metadata.get("file_name", "unknown"),
            "file_path": doc.metadata.get("file_path", ""),
            "file_type": doc.metadata.get("file_type", "")
        }
    )
    
    # Mostra progresso a cada 2 documentos
    if i % 2 == 0:
        print(f"   Processados {i}/{len(documentos)}...")

# Constrói o índice (isso pode demorar dependendo do volume)
builder.build_index(INDEX_PATH)

print(f"✅ Base vetorial criada em: {INDEX_PATH}")
print(f"📊 Economia de espaço: ~97% comparado ao FAISS")

📄 Encontrados 490 documentos
   Processados 2/490...
   Processados 4/490...
   Processados 6/490...
   Processados 8/490...
   Processados 10/490...
   Processados 12/490...
   Processados 14/490...
   Processados 16/490...
   Processados 18/490...
   Processados 20/490...
   Processados 22/490...
   Processados 24/490...
   Processados 26/490...
   Processados 28/490...
   Processados 30/490...
   Processados 32/490...
   Processados 34/490...
   Processados 36/490...
   Processados 38/490...
   Processados 40/490...
   Processados 42/490...
   Processados 44/490...
   Processados 46/490...
   Processados 48/490...
   Processados 50/490...
   Processados 52/490...
   Processados 54/490...
   Processados 56/490...
   Processados 58/490...
   Processados 60/490...
   Processados 62/490...
   Processados 64/490...
   Processados 66/490...
   Processados 68/490...
   Processados 70/490...
   Processados 72/490...
   Processados 74/490...
   Processados 76/490...
   Processados 78/490...


Writing passages: 100%|██████████| 478/478 [00:00<00:00, 27811.53chunk/s]
WARNING - leann.embedding_compute - Text 477 truncated: 4433 → 2048 tokens (2385 tokens removed)

WARNING - leann.embedding_compute - Text 477 truncated: 4433 → 2048 tokens (2385 tokens removed)
WARNING - leann.embedding_compute - Text 478 truncated: 55194 → 2048 tokens (53146 tokens removed)
WARNING - leann.embedding_compute - Truncation summary: 2/478 texts truncated (removed 55531 tokens total, longest was 55194 tokens)
Computing Ollama embeddings (batched):   0%|          | 0/4 [00:00<?, ?it/s]WARNING - leann.embedding_compute - Text 478 truncated: 55194 → 2048 tokens (53146 tokens removed)
WARNING - leann.embedding_compute - Truncation summary: 2/478 texts truncated (removed 55531 tokens total, longest was 55194 tokens)
Computing Ollama embeddings (batched): 100%|██████████| 4/4 [00:15<00:00,  3.79s/it]

M: 64 for level: 0
Starting conversion: leann_index.leann/leann_index.index -> leann_index.leann/leann_index.csr.tmp
[0.00s] Reading Index HNSW header...
[0.00s]   Header read: d=768, ntotal=478
[0.00s] Reading HNSW struct vectors...
  Reading vector (dtype=<class 'numpy.float64'>, fmt='d')... Count=6, Bytes=48
[0.00s]   Read assign_probas (6)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=6, Bytes=48
[0.00s]   Read assign_probas (6)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=7, Bytes=28
[0.15s]   Read cum_nneighbor_per_level (7)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=7, Bytes=28
[0.15s]   Read cum_nneighbor_per_level (7)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... 

Count=478, Bytes=1912
[0.27s]   Read levels (478)
[0.40s]   Probing for compact storage flag...
[0.40s]   Found compact flag: False
[0.40s]   Compact flag is False, reading original format...
[0.40s]   Probing for potential extra byte before non-compact offsets...
[0.40s]   Found and consumed an unexpected 0x00 byte.
  Reading vector (dtype=<class 'numpy.uint64'>, fmt='Q')... Count=479, Bytes=3832
[0.40s]   Read offsets (479)
[0.52s]   Attempting to read neighbors vector...
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=479, Bytes=3832
[0.40s]   Read offsets (479)
[0.52s]   Attempting to read neighbors vector...
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=30912, Bytes=123648
[0.52s]   Read neighbors (30912)
Count=30912, Bytes=123648
[0.52s]   Read neighbors (30912)
[0.64s]   Read scalar params (ep=473, max_lvl=1)
[0.64s] Checking for storage data...
[0.64s]   Found storage fourcc: 49467849.
[0.64s] Converting to CSR format...
[0.64s]   Conversion 

In [4]:
# ============================================================
# CHUNK 3 OTIMIZADO: VALIDAÇÃO COM LLM LOCAL (OLLAMA)
# VERSÃO CORRIGIDA - COM TRUNCAMENTO E PROMPT OTIMIZADO
# ============================================================

import pandas as pd
import numpy as np
import ollama
import json
import time
import re
import warnings
from concurrent.futures import ThreadPoolExecutor
from leann import LeannSearcher
warnings.filterwarnings('ignore')

print("="*80)
print("VALIDAÇÃO DE SIMILARIDADE COM LLM LOCAL (OLLAMA) - OTIMIZADO")
print("="*80)

# Carregar os dados
print("\nCarregando dados...")
eletr_2021_proc = pd.read_csv('eletr_2021_proc.csv', sep=';', encoding='utf-8-sig')
eletr_2021_proc_similar = pd.read_csv('eletr_2021_proc_similar.csv', sep=';', encoding='utf-8-sig')

print(f"✓ Projetos carregados: {len(eletr_2021_proc)}")
print(f"✓ Pares similares carregados: {len(eletr_2021_proc_similar)}")

# ============================================================
# CONFIGURAÇÃO
# ============================================================

MODEL = "qwen3:30b"
MAX_CHARS_CAMPO = 1500  # Aumentado - temos contexto de sobra
MAX_CHARS_LEANN = 4000  # Aumentado para mais contexto de referência
NUM_PREDICT = 1500  # Aumentado para respostas mais completas
BATCH_SIZE = 10  # Reduzido para melhor controle
MAX_TOTAL_CHARS = 80000  # Limite total para evitar estouro

print(f"\n⚙️ Configuração:")
print(f"   Modelo: {MODEL}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Max chars por campo: {MAX_CHARS_CAMPO}")
print(f"   Max chars LeANN: {MAX_CHARS_LEANN}")
print(f"   num_predict: {NUM_PREDICT}")

# ============================================================
# INICIALIZAR LEANN
# ============================================================

try:
    leann_searcher = LeannSearcher(index_path="./leann_index.leann/leann_index")
    print("✓ LeANN Searcher inicializado")
except Exception as e:
    print(f"⚠️ LeANN não disponível: {e}")
    leann_searcher = None

# ============================================================
# FUNÇÕES AUXILIARES COM TRUNCAMENTO
# ============================================================

def truncar_texto(texto, max_chars=800):
    """Trunca texto preservando palavras completas."""
    if pd.isna(texto) or not texto:
        return "Não informado"
    texto = str(texto).strip()
    texto = ' '.join(texto.split())  # Normaliza espaços
    if len(texto) <= max_chars:
        return texto
    # Trunca na última palavra completa
    truncado = texto[:max_chars].rsplit(' ', 1)[0]
    return truncado + "..."

def buscar_contexto_inovacao(searcher, texto: str, top_k: int = 3, max_chars: int = 2000) -> str:
    """Busca contexto LeANN com limite de tamanho."""
    if searcher is None:
        return ""
    try:
        resultados = searcher.search(texto[:500], top_k=top_k)  # Limita texto de busca
        contextos = []
        total_chars = 0
        for i, resultado in enumerate(resultados, 1):
            texto_resultado = resultado.text[:600]  # Limita cada resultado
            if total_chars + len(texto_resultado) > max_chars:
                break
            contextos.append(f"[{i}] {texto_resultado}")
            total_chars += len(texto_resultado)
        return "\n".join(contextos) if contextos else ""
    except:
        return ""

def extrair_info_projeto(projeto_row: pd.Series, max_chars: int = 800) -> dict:
    """Extrai informações do projeto com truncamento."""
    campos = ["nome_projeto", "elemento_tecnologico", "desafio_tecnologico", "metodologia"]
    info = {}
    for campo in campos:
        if campo in projeto_row.index:
            info[campo] = truncar_texto(projeto_row[campo], max_chars)
        else:
            info[campo] = "Não informado"
    return info

# ============================================================
# SYSTEM PROMPT SIMPLIFICADO E DIRETO
# ============================================================

SYSTEM_PROMPT_BASE = """Você é um especialista em avaliação de projetos de P&D para enquadramento na Lei do Bem (Lei nº 11.196/2005).

## CONTEXTO LEGAL E TÉCNICO

A Lei do Bem oferece incentivos fiscais para empresas que investem em Pesquisa e Desenvolvimento tecnológico. Para um projeto ser elegível, deve demonstrar:

### 1. REQUISITOS FUNDAMENTAIS DE P&D (Manual de Frascati - OCDE)

**Pesquisa Básica:** Trabalho experimental ou teórico para adquirir novos conhecimentos sobre fenômenos e fatos observáveis, sem aplicação ou uso particular em vista.

**Pesquisa Aplicada:** Investigação original para adquirir novos conhecimentos, dirigida a objetivo prático específico.

**Desenvolvimento Experimental:** Trabalho sistemático baseado em conhecimentos existentes, visando produção de novos materiais, produtos, dispositivos, processos, sistemas ou serviços, ou melhoria substancial dos já existentes.

### 2. CRITÉRIOS DE ENQUADRAMENTO (5 requisitos obrigatórios)

1. **NOVIDADE:** O projeto deve buscar novo conhecimento ou aplicação não trivial
2. **CRIATIVIDADE:** Deve haver conceitos e hipóteses originais, não óbvios para profissional da área
3. **INCERTEZA TECNOLÓGICA:** O resultado não é previsível com base no estado da arte
4. **SISTEMATICIDADE:** Metodologia estruturada com objetivos, cronograma e recursos definidos
5. **TRANSFERIBILIDADE:** Resultados devem ser reproduzíveis ou codificáveis

### 3. O QUE NÃO É P&D (exclusões importantes)

- Melhorias rotineiras ou incrementais de produtos/processos existentes
- Customização de software ou adaptação para cliente específico
- Treinamento de pessoal ou implantação de sistemas comerciais
- Atividades de engenharia de produção ou controle de qualidade rotineiro
- Prospecção, exploração ou perfuração de petróleo/gás
- Mudanças estéticas ou de design sem avanço tecnológico

### 4. ELEMENTOS DE ANÁLISE PARA COMPARAÇÃO

**ELEMENTO TECNOLÓGICO:** Qual tecnologia, método ou sistema está sendo desenvolvido/aprimorado?
- Hardware, software, processo produtivo, material, algoritmo, etc.

**DESAFIO TECNOLÓGICO:** Qual problema técnico-científico está sendo resolvido?
- Qual a incerteza? O que não se sabe fazer ou não existe no mercado?

**METODOLOGIA:** Como o problema será resolvido?
- Quais etapas de pesquisa? Há experimentação? Há validação?

---

## SUA TAREFA: COMPARAR DOIS PROJETOS

Analise os dois projetos e determine se são:
- O mesmo projeto (possível duplicação/plágio)
- Projetos relacionados (mesmo tema, diferentes abordagens)
- Projetos independentes (sem relação significativa)

### CRITÉRIOS DE CLASSIFICAÇÃO

| Classificação | Definição | Indicadores |
|---------------|-----------|-------------|
| **IDÊNTICOS** | Textos praticamente iguais | Mesmo elemento, mesmo desafio, mesma metodologia. Diferenças apenas cosméticas (sinônimos, ordem de palavras). Possível duplicação ou cópia. |
| **MUITO_SIMILARES** | Mesmo núcleo de inovação | Mesmo domínio tecnológico e problema central. Metodologias muito parecidas. Variações menores (ex: parâmetros diferentes, escopo ligeiramente diferente). |
| **SIMILARES** | Mesma área tecnológica | Tecnologias relacionadas mas elementos distintos. Desafios no mesmo campo mas problemas diferentes. Metodologias com algumas etapas em comum. |
| **DISTINTOS** | Sem relação significativa | Elementos tecnológicos claramente diferentes. Desafios sem conexão. Metodologias independentes. Áreas ou domínios diferentes. |

### REGRAS OBRIGATÓRIAS PARA "mesmo_tratamento"

| Classificação | mesmo_tratamento | Justificativa |
|---------------|------------------|---------------|
| IDÊNTICOS | **true** (OBRIGATÓRIO) | Projetos idênticos DEVEM receber tratamento igual - é logicamente impossível tratá-los diferente |
| DISTINTOS | **false** (OBRIGATÓRIO) | Projetos diferentes DEVEM ser avaliados independentemente |
| MUITO_SIMILARES | Depende da análise | true: se compartilham mesmo núcleo de inovação; false: se cada um tem mérito próprio |
| SIMILARES | Depende da análise | true: se risco de serem mesmo projeto; false: se genuinamente distintos |

### SINAIS DE ALERTA (alerta_duplicacao = true)

- Textos com mais de 80% de sobreposição
- Mesmas frases ou parágrafos repetidos
- Descrições técnicas idênticas com nomes de empresa diferentes
- Projetos de empresas diferentes com elementos tecnológicos idênticos

---

## FORMATO DE RESPOSTA

Responda EXCLUSIVAMENTE com o JSON abaixo, sem texto adicional antes ou depois:

{
    "classificacao": "IDÊNTICOS|MUITO_SIMILARES|SIMILARES|DISTINTOS",
    "mesmo_tratamento": true|false,
    "justificativa": "Comparação específica dos elementos tecnológicos, desafios e metodologias dos dois projetos",
    "alerta_duplicacao": true|false
}

IMPORTANTE: 
- Não inclua explicações fora do JSON
- Não use markdown ou formatação adicional
- Justifique sempre comparando explicitamente os 3 elementos: tecnológico, desafio e metodologia"""

# ============================================================
# FUNÇÃO DE ANÁLISE OTIMIZADA
# ============================================================

def analisar_par_projetos(proj1_data, proj2_data, empresa1, empresa2, par_id, id_proj1, id_proj2):
    """Analisa um par de projetos com prompt otimizado."""
    
    # Extrair info com truncamento
    info1 = extrair_info_projeto(proj1_data, MAX_CHARS_CAMPO)
    info2 = extrair_info_projeto(proj2_data, MAX_CHARS_CAMPO)
    
    # Buscar contexto LeANN (limitado)
    texto_busca = f"{info1.get('elemento_tecnologico', '')} {info2.get('elemento_tecnologico', '')}"
    contexto_leann = buscar_contexto_inovacao(leann_searcher, texto_busca, top_k=2, max_chars=MAX_CHARS_LEANN)
    
    # System prompt com contexto opcional
    system_prompt = SYSTEM_PROMPT_BASE
    if contexto_leann:
        system_prompt += f"\n\nCONTEXTO DE REFERÊNCIA:\n{contexto_leann}"
    
    # User prompt compacto
    user_prompt = f"""PROJETO 1 ({empresa1[:30]}):
- Nome: {info1['nome_projeto'][:100]}
- Elemento: {info1['elemento_tecnologico']}
- Desafio: {info1['desafio_tecnologico']}

PROJETO 2 ({empresa2[:30]}):
- Nome: {info2['nome_projeto'][:100]}
- Elemento: {info2['elemento_tecnologico']}
- Desafio: {info2['desafio_tecnologico']}

Responda APENAS com o JSON especificado."""
    
    try:
        response = ollama.chat(
            model=MODEL,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_prompt}
            ],
            options={
                'temperature': 0.1,
                'num_predict': NUM_PREDICT,
            }
        )
        
        llm_response = response['message']['content']
        
        # Limpeza robusta da resposta
        llm_response = re.sub(r'<think>.*?</think>', '', llm_response, flags=re.DOTALL)
        llm_response = llm_response.replace('```json', '').replace('```', '').strip()
        
        # Extrair JSON
        json_match = re.search(r'\{\s*"classificacao".*?\}', llm_response, re.DOTALL)
        
        if not json_match:
            # Tentar encontrar qualquer JSON válido
            json_match = re.search(r'\{[^{}]*"classificacao"[^{}]*\}', llm_response, re.DOTALL)
        
        if json_match:
            json_str = json_match.group()
            parsed = json.loads(json_str)
            
            # Normalizar classificação
            classificacao = str(parsed.get('classificacao', '')).upper().strip()
            classificacao = classificacao.replace('Ê', 'E').replace('Í', 'I')
            
            # Aplicar regras obrigatórias
            if 'IDENT' in classificacao or classificacao == 'IDENTICOS':
                parsed['classificacao'] = 'IDÊNTICOS'
                parsed['mesmo_tratamento'] = True
            elif 'DISTINT' in classificacao:
                parsed['classificacao'] = 'DISTINTOS'
                parsed['mesmo_tratamento'] = False
            elif 'MUITO' in classificacao:
                parsed['classificacao'] = 'MUITO_SIMILARES'
            elif 'SIMILAR' in classificacao:
                parsed['classificacao'] = 'SIMILARES'
            
            parsed['par_id'] = par_id
            parsed['id_projeto_1'] = id_proj1
            parsed['id_projeto_2'] = id_proj2
            return parsed
        else:
            return {
                "erro": "JSON não encontrado",
                "par_id": par_id,
                "id_projeto_1": id_proj1,
                "id_projeto_2": id_proj2,
                "resposta_raw": llm_response[:200]
            }
            
    except json.JSONDecodeError as e:
        return {
            "erro": f"JSON inválido: {str(e)[:50]}",
            "par_id": par_id,
            "id_projeto_1": id_proj1,
            "id_projeto_2": id_proj2
        }
    except Exception as e:
        return {
            "erro": f"Exceção: {str(e)[:50]}",
            "par_id": par_id,
            "id_projeto_1": id_proj1,
            "id_projeto_2": id_proj2
        }

# ============================================================
# PROCESSAMENTO EM LOTES
# ============================================================

def processar_todos_pares():
    """Processa todos os pares de forma otimizada."""
    
    print("\n" + "="*80)
    print("INICIANDO ANÁLISE")
    print("="*80)
    
    # Criar índice por id_projeto
    proj_dict = eletr_2021_proc.set_index('id_projeto').to_dict('index')
    
    total_pares = len(eletr_2021_proc_similar)
    print(f"\n📊 Total de pares: {total_pares}")
    
    todos_resultados = []
    erros = 0
    sucessos = 0
    
    for idx, row in eletr_2021_proc_similar.iterrows():
        id_proj1 = row['IdProjeto_1']
        id_proj2 = row['IdProjeto_2']
        
        # Buscar dados dos projetos
        if id_proj1 not in proj_dict or id_proj2 not in proj_dict:
            print(f"  ⚠️ Par {idx}: Projeto não encontrado")
            continue
        
        proj1_data = pd.Series(proj_dict[id_proj1])
        proj2_data = pd.Series(proj_dict[id_proj2])
        
        empresa1 = str(row.get('Empresa_1', 'Empresa 1'))
        empresa2 = str(row.get('Empresa_2', 'Empresa 2'))
        
        # Analisar par
        resultado = analisar_par_projetos(
            proj1_data, proj2_data,
            empresa1, empresa2,
            idx, id_proj1, id_proj2
        )
        
        # Adicionar info do par
        resultado['nome_projeto_1'] = row['NomeProjeto_1']
        resultado['nome_projeto_2'] = row['NomeProjeto_2']
        resultado['similaridade_coseno'] = row['Similaridade_score']
        resultado['mesma_empresa'] = row.get('Mesma_Empresa', False)
        
        todos_resultados.append(resultado)
        
        # Log
        if 'erro' in resultado:
            erros += 1
            print(f"  ❌ Par {id_proj1} x {id_proj2}: {resultado['erro'][:40]}")
        else:
            sucessos += 1
            classif = resultado.get('classificacao', 'N/A')
            mesmo = "✓" if resultado.get('mesmo_tratamento', False) else "✗"
            print(f"  ✓ Par {id_proj1} x {id_proj2}: {classif} | Mesmo: {mesmo}")
        
        # Progresso a cada 20 pares
        if (idx + 1) % 20 == 0:
            print(f"\n  📊 Progresso: {idx+1}/{total_pares} | Sucessos: {sucessos} | Erros: {erros}\n")
        
        # Pausa para não sobrecarregar
        time.sleep(0.5)
    
    return todos_resultados

# ============================================================
# EXECUTAR
# ============================================================

print("\nIniciando processamento...")
inicio = time.time()

resultados = processar_todos_pares()

tempo_total = time.time() - inicio

# ============================================================
# ANÁLISE DOS RESULTADOS
# ============================================================

print("\n" + "="*80)
print("RESULTADOS")
print("="*80)

df_resultados = pd.DataFrame(resultados)

total = len(df_resultados)
com_erro = df_resultados['erro'].notna().sum() if 'erro' in df_resultados.columns else 0
sem_erro = total - com_erro

print(f"\n📊 Total: {total}")
print(f"   ✓ Sucesso: {sem_erro} ({sem_erro/total*100:.1f}%)")
print(f"   ❌ Erros: {com_erro} ({com_erro/total*100:.1f}%)")
print(f"   ⏱️ Tempo: {tempo_total/60:.1f} min")

if 'classificacao' in df_resultados.columns:
    df_validos = df_resultados[df_resultados['classificacao'].notna()]
    print(f"\n📋 Distribuição:")
    print(df_validos['classificacao'].value_counts())
    
    if 'mesmo_tratamento' in df_validos.columns:
        mesmo = df_validos['mesmo_tratamento'].sum()
        print(f"\n📌 Mesmo tratamento: {mesmo}/{len(df_validos)} ({mesmo/len(df_validos)*100:.1f}%)")

# Salvar
arquivo = 'validacao_llm_pares_similares.csv'
df_resultados.to_csv(arquivo, index=False, encoding='utf-8-sig', sep=';')
print(f"\n✓ Salvo em: {arquivo}")

print("\n" + "="*80)
print("CONCLUÍDO")
print("="*80)

VALIDAÇÃO DE SIMILARIDADE COM LLM LOCAL (OLLAMA) - OTIMIZADO

Carregando dados...
✓ Projetos carregados: 366
✓ Pares similares carregados: 70

⚙️ Configuração:
   Modelo: qwen3:30b
   Batch size: 10
   Max chars por campo: 1500
   Max chars LeANN: 4000
   num_predict: 1500
[read_HNSW - CSR NL v4] Reading metadata & CSR indices (manual offset)...
[read_HNSW NL v4] Read levels vector, size: 478
[read_HNSW NL v4] Reading Compact Storage format indices...
[read_HNSW NL v4] Read compact_level_ptr, size: 966
[read_HNSW NL v4] Read compact_node_offsets, size: 479
[read_HNSW NL v4] Read entry_point: 473, max_level: 1
[read_HNSW NL v4] Read storage fourcc: 0x6c6c756e
[read_HNSW NL v4 FIX] Detected FileIOReader. Neighbors size field offset: 13650
[read_HNSW NL v4] Reading neighbors data into memory.
[read_HNSW NL v4] Read neighbors data, size: 6555
[read_HNSW NL v4] Finished reading metadata and CSR indices.
INFO: Skipping external storage loading, since is_recompute is true.
✓ LeANN Searcher in

[HNSW RNG] get_vector_zmq id=473 cache_hit=0
Computing Ollama embeddings (batched): 100%|██████████| 1/1 [00:00<00:00,  2.10it/s]


  ✓ Par 68931 x 68967: IDÊNTICOS | Mesmo: ✓
ZmqDistanceComputer initialized: d=768, metric=0
ZmqDistanceComputer initialized: d=768, metric=0


[HNSW RNG] get_vector_zmq id=473 cache_hit=0
Computing Ollama embeddings (batched): 100%|██████████| 1/1 [00:00<00:00,  2.37it/s]


  ✓ Par 68304 x 68315: IDÊNTICOS | Mesmo: ✓
ZmqDistanceComputer initialized: d=768, metric=0
ZmqDistanceComputer initialized: d=768, metric=0


...

  ✓ Par 56363 x 67366: MUITO_SIMILARES | Mesmo: ✓
ZmqDistanceComputer initialized: d=768, metric=0
ZmqDistanceComputer initialized: d=768, metric=0


[HNSW RNG] get_vector_zmq id=473 cache_hit=0
Computing Ollama embeddings (batched): 100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


  ✓ Par 56363 x 56364: MUITO_SIMILARES | Mesmo: ✓
ZmqDistanceComputer initialized: d=768, metric=0
ZmqDistanceComputer initialized: d=768, metric=0


[HNSW RNG] get_vector_zmq id=473 cache_hit=0
Computing Ollama embeddings (batched): 100%|██████████| 1/1 [00:00<00:00,  2.72it/s]


  ✓ Par 68467 x 68512: MUITO_SIMILARES | Mesmo: ✓

RESULTADOS

📊 Total: 70
   ✓ Sucesso: 70 (100.0%)
   ❌ Erros: 0 (0.0%)
   ⏱️ Tempo: 27.8 min

📋 Distribuição:
classificacao
IDÊNTICOS          37
MUITO_SIMILARES    33
Name: count, dtype: int64

📌 Mesmo tratamento: 70/70 (100.0%)

✓ Salvo em: validacao_llm_pares_similares.csv

CONCLUÍDO

RESULTADOS

📊 Total: 70
   ✓ Sucesso: 70 (100.0%)
   ❌ Erros: 0 (0.0%)
   ⏱️ Tempo: 27.8 min

📋 Distribuição:
classificacao
IDÊNTICOS          37
MUITO_SIMILARES    33
Name: count, dtype: int64

📌 Mesmo tratamento: 70/70 (100.0%)

✓ Salvo em: validacao_llm_pares_similares.csv

CONCLUÍDO
